In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import torch
import pandas as pd
from pathlib import Path
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import multiprocessing
from langchain_community.chat_models import ChatLlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# RAG 체인 구성
from langchain.chains import RetrievalQA



print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print((torch.cuda._get_nvml_device_index(0)))

True
NVIDIA A100-SXM4-80GB
3


In [3]:
# Path 객체에는 -r 옵션(재귀적 검색)이 없습니다.
# 만약 특정 디렉토리 내에서 재귀적으로 파일을 찾고 싶다면 glob 메서드나 rglob 메서드를 사용해야 합니다.
# 아래는 예시입니다.

root_path = Path(".").resolve().parent.parent
model_path = root_path / "ai_models" / "base_models" / "BGE-m3-ko"
print(model_path)

/AIHUB/PCNRND/home/chatbot/ai_models/base_models/BGE-m3-ko


# 

In [4]:
# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name=str(model_path),
    model_kwargs={'device': 'cuda:0'},
    encode_kwargs={'normalize_embeddings': True}
)

/tmp/ipykernel_3416927/3262055005.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
faiss_index_directory = "./data/faiss_csv_db"
vectorstore = FAISS.load_local(faiss_index_directory, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(    
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5, "k": 3}
)

In [6]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f7f2f735590>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.5, 'k': 3})

In [7]:
llm_path = str(root_path / "ai_models" / "gguf_models" / "MLP-KTLim" / "llama-3-Korean-Bllossom-8B-gguf-Q4_K_M" / "llama-3-Korean-Bllossom-8B-Q4_K_M.gguf")

print(llm_path)

/AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf


In [8]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# RAG 시스템을 위한 LLM 설정 (KV 캐시 최적화 포함)
llm = ChatLlamaCpp(
    temperature=0.1,  # RAG에서는 더 일관된 답변을 위해 낮은 temperature 사용
    model_path=llm_path,  # PosixPath를 문자열로 변환
    n_ctx=8192,  # 컨텍스트 크기를 적절히 조정 (32768은 메모리 부담이 큼)
    n_gpu_layers=8,
    n_batch=64,  # KV 캐시 최적화를 위해 더 작은 배치 크기
    max_tokens=512,  # 더 긴 답변을 위해 토큰 수 증가
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.1,  # RAG에서는 적당한 반복 패널티
    top_p=0.9,  # RAG에서는 더 높은 top_p로 다양한 답변 생성
    verbose=False,  # RAG에서는 verbose 비활성화로 출력 정리
    callback_manager=callback_manager,
    
    # KV 캐시 최적화를 위한 설정들
    use_mlock=True,  # 메모리 잠금으로 성능 향상
    use_mmap=True,   # 메모리 맵핑 사용
)

In [9]:
llm

ChatLlamaCpp(verbose=False, callbacks=<langchain_core.callbacks.manager.CallbackManager object at 0x7f7f2e756590>, client=<llama_cpp.llama.Llama object at 0x7f7f2f733850>, model_path='/AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf', n_ctx=8192, use_mlock=True, n_threads=127, n_batch=64, n_gpu_layers=8, max_tokens=512, temperature=0.1, top_p=0.9, model_kwargs={})

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False
)

In [11]:
# 프롬프트 구성
question = "피씨엔 연구과제에 대해 알려줘"
prompt = f"<|im_start|>system\n당신은 도움이 되는 AI 어시스턴트입니다. 질문에 대해 3문장 정도 정확하고 유용한 답변을 제공해주세요.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"

In [12]:
response = qa_chain.invoke(prompt)

안녕하세요! 피씨엔의 연구과제에 대해 알려드릴게요.

피씨엔은 빅데이터와 인공지능(AI)을 활용한 다양한 연구과제를 수행하고 있습니다. 주요 연구과제는 다음과 같습니다:

1. **산업 데이터 상태 진단 및 개선 극대화를 위한 증강분석 기반 자동 데이터 시멘틱 정보 구성/표현 기술 개발**

이 과제는 산업 데이터의 상태를 진단하고, 이를 개선하기 위해 증강분석 기법을 활용하는 것입니다.

2. **인공지능 비전(AI VISION)기반, 데이터 증강 및 결함탐지 플랫폼 개발**

이 과제는 인공지능 기술을 활용하여, 데이터의 시각적 표현을 강화하고, 데이터의 결함을 탐지할

KeyboardInterrupt: 

In [13]:
from sentence_transformers import CrossEncoder

In [14]:
# 2) 1차 검색기로부터 top_k 문서 가져오기
top_k = 20
retrieved_docs = vectorstore.similarity_search(question, k=top_k)

In [15]:
# 3) CrossEncoder 기반 Re-ranker 준비
reranker = CrossEncoder(model_path)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at /AIHUB/PCNRND/home/chatbot/ai_models/base_models/BGE-m3-ko and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# 4) (query, passage) 쌍으로 점수 계산
pairs = [(question, doc.page_content) for doc in retrieved_docs]
scores = reranker.predict(pairs)

# 5) 점수를 기준으로 정렬
scored_docs = list(zip(retrieved_docs, scores))
scored_docs.sort(key=lambda x: x[1], reverse=True)

In [18]:
# 6) 상위 N개 선택
top_n = 5
final_docs = [doc for doc, score in scored_docs[:top_n]]

In [25]:
final_docs

[Document(id='0c35d189-2ee6-4793-9253-1a725d01676a', metadata={'source': './PCN_지식재산권현황.csv', 'row': 24}, page_content='출원: 등록\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 빅데이터 증강분석 프로파일링 시스템\n국가명: 대한민국\n출원ㆍ등록번호/출원ㆍ등록일: 10-2541934/2023-06-05\n비고: \n상태: 등록확인'),
 Document(id='4dfe4993-9f4c-49a9-b6f2-cfaf874c7a41', metadata={'source': './PCN_지식재산권현황.csv', 'row': 25}, page_content='출원: 등록\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 빅데이터 증강분석 프로파일링 시스템\n국가명: PCT/국제\n출원ㆍ등록번호/출원ㆍ등록일: PCT/KR2021/000685\n비고: \n상태: 진행중'),
 Document(id='f78e1ca8-f387-46ab-a17a-b913a814e2c5', metadata={'source': './PCN_지식재산권현황.csv', 'row': 2}, page_content='출원: 출원\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 민군 인공위성 궤도, 충돌 및 추락정보 연동을 통한 실시간 궤도 비교 분석, 충돌 및 추락 시뮬레이션시스템\n국가명: 대한민국\n출원ㆍ등록번호/출원ㆍ등록일: 10-2024-187439\n비고: \n상태: 진행중'),
 Document(id='4c78bec4-60fa-42de-a78b-3cbce66856af', metadata={'source': './PCN_지식재산권현황.csv', 'row': 3}, page_content='출원: 출원\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 멀티 클라우드 환경에서의 클라우드 네이티브 기반 실시간 XR 클라우드 컴퓨팅 시스템\n국가명: 대한민국\n출원ㆍ등록번호/출원ㆍ등록일: 10-

In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": top_n})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # 로컬 LLM이면 교체
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
context = "\n\n".join([doc.page_content for doc in final_docs])
prompt = f"다음 문서를 참고하여 질문에 답변해줘:\n\n{context}\n\n질문: {question}"
result = qa_chain.invoke(prompt)

  **
 
 
 
 
 
 
 
  # 1.  - 
 
 
 
 
  --  -- 
 
 
 
 
 
 or  -  
 ` 
 
 to the  - a  -  - 
 
 
  - 
 
 
  ?  : 0/ 
 10/ 1/ 9 1 1 out your question the ` 

 
 
 
  you need what ` 
 
 
 
 'your ` 
 
 and out ` 
 
 
 
 up or \\ 
 out 2 `  
 # 0 out out out out out out out out  -- out of the out out out out out the out the 1/5 out your 10
 
 
 
 
 
 
 
 
 
 ``` 
 ... 1/만/ – 
 out out 그 다 9, 10/정 1/시 - 

  
 
 고 ` 
 ``` \to out the out out the `  
기- 
 1. 0/정 세/정 선 `이 프
### ` 
 ` 
 10/정 '상
## 10/ 10/정 이 1/10 or 또는 다/9/7/정 `그 포 10/9/ 